# 교차 검증과 그리드 서치
#### 훈련셋으로 훈련하고, 테스트셋으로 테스트하면서 과대, 과소적합 해결과 편향을 줄일 수 있음
#### 하지만, 지속적인 테스트셋의 결과로 인한 수정은, 결국 테스트셋에 잘 맞는 모델이 만들어짐
## 검증 세트

In [25]:
import pandas as pd
import numpy as np

In [26]:
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [27]:
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [28]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [31]:
# DataSet: Train-Validation 분할
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [32]:
# 3개 데이터셋의 형태 확인
print(sub_input.shape, val_input.shape, test_input.shape)

(4157, 3) (1040, 3) (1300, 3)


In [33]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


### 교차 검증

In [34]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target) # 5폴드
print(scores)

{'fit_time': array([0.00598669, 0.00698471, 0.00698543, 0.0069809 , 0.00598836]), 'score_time': array([0.00099397, 0.00099373, 0.        , 0.00099325, 0.00099659]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [35]:
# 출력값의 평균
print(np.mean(scores['test_score']))

0.855300214703487


In [36]:
### cross_validate는 분류 문제일 경우, StratifiedkFold를 사용
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold()) # 5폴드
print(np.mean(scores['test_score']))

0.855300214703487


In [37]:
# 훈련셋을 셔플하거나 폴드 수를 정하는 파라미터
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


# 하이퍼 파라미터 튜닝

#### 모델이 학습한 파라미터 = 모델 파라미터(coef_, intercept_등)
#### 모델이 직접 설정할 수 없어서 개발자가 지정하는 파라미터 = 하이퍼 파라미터

In [38]:
from sklearn.model_selection import GridSearchCV

In [39]:
# min_impurity_decrease = 트리 모델의 depth를 지니 불순도의 변화량에 따라 분할할지 안할지 결정하는 파라미터
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [40]:
# GridSearch 객체 생성
# GridSearchCV(모델, 딕셔너리, 코어)
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [41]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [43]:
# dt객체 생성
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

0.9615162593804117
0.8653846153846154


In [44]:
# 그리드 객체의 모델 파라미터 중 best_params_는 그리드 서치가 탐색한 최적의 파라미터를 반환
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [45]:
# 검증셋으로 얻은 결과값 확인
gs.cv_results_

{'mean_fit_time': array([0.00877533, 0.00817704, 0.0067812 , 0.00678186, 0.00518498]),
 'std_fit_time': array([0.00074631, 0.00097694, 0.00116301, 0.00097719, 0.00074614]),
 'mean_score_time': array([0.00159626, 0.001197  , 0.00159607, 0.00119748, 0.00099754]),
 'std_score_time': array([4.89142299e-04, 7.46442197e-04, 4.88986813e-04, 3.99399316e-04,
        6.91002691e-07]),
 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_impurity_decrease': 0.0001},
  {'min_impurity_decrease': 0.0002},
  {'min_impurity_decrease': 0.0003},
  {'min_impurity_decrease': 0.0004},
  {'min_impurity_decrease': 0.0005}],
 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077, 0.86538462]),
 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154, 0.86923077]),
 'split2_test_score': array([0.8825794 , 0.8

In [47]:
# 그리드 객체의 cv_results 검증결과 안의 5개의 min_inpurity 각각의 평균값
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [49]:
# mean_test_score 중 최대값과 gs.best_params_는 동일함
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [52]:
# 여러개의 파라미터 설정
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }
print(len(np.arange(0.0001, 0.001, 0.0001)) * len(range(5, 20, 1)) * len(range(2, 100, 10)) * 5, '개의 모델을 학습합니다')

6750 개의 모델을 학습합니다


In [53]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [54]:
# 베스트 파라미터 출력
gs.best_params_

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

In [55]:
gs.cv_results_

{'mean_fit_time': array([0.00558519, 0.00478649, 0.00598431, ..., 0.00538554, 0.00478735,
        0.004387  ]),
 'std_fit_time': array([0.00079811, 0.00074661, 0.00089154, ..., 0.00135405, 0.00074626,
        0.00048912]),
 'mean_score_time': array([0.00159602, 0.00199556, 0.0019949 , ..., 0.00059829, 0.00099707,
        0.00039892]),
 'std_score_time': array([4.88461088e-04, 6.30676406e-04, 6.31808234e-04, ...,
        4.88500751e-04, 1.50789149e-07, 4.88577656e-04]),
 'param_max_depth': masked_array(data=[5, 5, 5, ..., 19, 19, 19],
              mask=[False, False, False, ..., False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0001, 0.0001, ..., 0.0009000000000000001,
                    0.0009000000000000001, 0.0009000000000000001],
              mask=[False, False, False, ..., False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_split': masked_array(data=[2, 

## 랜덤 서치

In [56]:
# 범위를 우리가 직접 정하기에는 너무 위험함
# 싸이 파이에서 2개의 확률분포 클래스를 임포트
from scipy.stats import uniform, randint

In [57]:
# 0~10 사이의 범위를 갖는 객체를 만들고 10개의 숫자를 샘플링
rgen = randint(0, 10)
rgen.rvs(10)

array([4, 8, 6, 6, 1, 2, 3, 8, 1, 1], dtype=int64)

In [58]:
# 고르게 추출 되는지 확인
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([ 88,  93, 108, 122,  96,  96, 114,  99,  80, 104], dtype=int64))

In [59]:
# uniform 적용
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.6881335 , 0.32665344, 0.62190354, 0.96286045, 0.33588933,
       0.09736814, 0.60833738, 0.12059226, 0.54388064, 0.5851768 ])

In [60]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25)
          }

In [61]:
from sklearn.model_selection import RandomizedSearchCV

In [62]:
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, # 위에서 활성화했던 rvs와 같은 의미
                        n_jobs=-1,
                        random_state=42)

rs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001ED4F4D1B90>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001ED4F2DEE50>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001ED4F4D3E50>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001ED4F4D3710>},
                   random_state=42)

In [63]:
rs.best_params_

{'max_depth': 39,
 'min_impurity_decrease': 0.00034102546602601173,
 'min_samples_leaf': 7,
 'min_samples_split': 13}

In [64]:
# 실제 모델에 적용
dt = rs.best_estimator_
print(dt.score(test_input, test_target))

0.86
